In [ ]:
import paddlehub as hub 
senta = hub.Module(name="senta_bilstm") 

In [ ]:
import pandas as pd 
import os
df = pd.read_excel('wom.xlsx') 
df = df.fillna('')
df = df.replace('\n','', regex=True)
df = df.replace('\u2002','', regex=True)

In [ ]:
CSC = pd.DataFrame()
for i in range(168):
    CourseName = df.iloc[i][0]
    TeacherName = df.iloc[i][1]
    
    for j in range(2,38):
        Comments = df.iloc[i][j]

        if Comments:
            t = [Comments]
            input_dict = {"text": t}
            results = senta.sentiment_classify(data=input_dict)[0]
            SentimentScore = results.get('positive_probs')
            if SentimentScore >= 0.6:
                SentimentLabel = '正面'
            elif SentimentScore >= 0.4 and SentimentScore < 0.6:
                SentimentLabel = '中立'
            else:
                SentimentLabel = '負面'
#             print(CourseName,TeacherName,Comments,SentimentScore,SentimentLabel,"\n")
            if Comments=='None':
                Results = pd.DataFrame(
                        data = [{'CourseName':CourseName,'TeacherName':TeacherName,'Comments':Comments,'SentimentScore':0,'SentimentLabel':'無口碑'}],
                        columns = ['CourseName', 'TeacherName', 'Comments', 'SentimentScore', 'SentimentLabel'])
                print(Results)
                CSC = pd.concat([CSC, Results])
            else:
                Results = pd.DataFrame(
                        data = [{'CourseName':CourseName,'TeacherName':TeacherName,'Comments':Comments,'SentimentScore':SentimentScore,'SentimentLabel':SentimentLabel}],
                        columns = ['CourseName', 'TeacherName', 'Comments', 'SentimentScore', 'SentimentLabel'])
                print(Results)
                CSC = pd.concat([CSC, Results])

In [ ]:
avg = CSC.groupby(['CourseName','TeacherName']).mean()
avg = avg.sort_values('SentimentScore', ascending=False)
wom = ''
a=[]
b=[]
c=[]
d=[]
e=[]
f=[]
for i in avg.iterrows():
    if 0.8 <= i[1][0]:
        a.append([i[0][0],i[0][1],i[1][0]])
    if 0.6 < i[1][0] < 0.8:
        b.append([i[0][0],i[0][1],i[1][0]])
    if 0.4 < i[1][0] < 0.6:
        c.append([i[0][0],i[0][1],i[1][0]])
    if 0.2 < i[1][0] < 0.4:
        d.append([i[0][0],i[0][1],i[1][0]])
    if [1][0] <= 0.2:
        e.append([i[0][0],i[0][1],i[1][0]])
    if i[1][0] == 0:
        f.append([i[0][0],i[0][1],i[1][0]])

print('\n極為正面 : ',len(a),'\n',a)
print('\n正面 : ',len(b),'\n',b)
print('\n褒貶不一 :',len(c),' \n',c)
print('\n負面 :',len(d),' ',d)
print('\n極為負面 :',len(e),'\n ',e)
print('\n無評論 :',len(f),'\n ',f)

In [ ]:
CSC.to_excel('CSC.xlsx', engine='xlsxwriter')

In [ ]:
avg = CSC.groupby(['CourseName','TeacherName']).mean()
avg = avg.sort_values('SentimentScore', ascending=False)

In [ ]:
avg.to_excel('avg.xlsx')

In [ ]:
import monpa

In [ ]:
CSC['Comments'].head(10)

In [ ]:
def LongPseg(long_sentence, split_char):
    seg = []
    for item in long_sentence.split(split_char):
        if item != "\n": seg.extend(monpa.pseg(item+split_char))
    return seg[:-1]

In [ ]:
from collections import Counter
WORD = []
for i in CSC['Comments']:
    result = LongPseg(i,"，")
    word = result[0][0]
    pos = result[0][1]
#     if pos== 'Na':
#         print(word)
#     if pos == 'Nb':
#         print(word)
#     if pos == 'VH':
#         print(word)
    for j in range(len(result)):
        word = result[j][0]
        WORD.append(word)

In [ ]:
RESULT = pd.DataFrame()
P = Counter(WORD).most_common()

for i in range(2730):
    p = monpa.pseg(P[i][0])
    if int(P[i][1]) >= 5:
        try:
            Text = P[i][0]
            Pos = p[0][1]
            Count = P[i][1]
#             print(Text,Pos,Count)
            RESULTS = pd.DataFrame(data = [{'Text':Text,
                                 'Pos':Pos,
                                 'Count':Count}],
                      columns = ['Text', 'Pos', 'Count'])
            RESULT = pd.concat([RESULT, RESULTS], ignore_index=True)
        except: 
            print('error')
# RESULT.to_excel('Pos_Count.xlsx')
RESULT

In [ ]:
RESULT.to_excel('Pos_Count.xlsx')

In [ ]:
Text = '低'
Pos = 'VH'
Count = '8'
RESULT = pd.DataFrame(data = [{'Text':Text,
                                 'Pos':Pos,
                                 'Count':Count}],
                      columns = ['Text', 'Pos', 'Count'])
TITLE = pd.DataFrame()
RESULT = pd.concat([TITLE, RESULT], ignore_index=True)

In [ ]:
RESULT